<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=17f19e1d07b18a1d3a411d88edfeb00091b04bb4f0ff11d24db59217d6b65480
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7


Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-29 11:46:59
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.46 C
-------------------
Today PnL: 83.97 K (0.58%)
Current PnL: -18.51 L (-12.13%)
CY Booked + Current PnL: -6.38 L (-4.18%)
-------------------
Total profit:  2.78 L
Total loss:  -21.29 L
-------------------
Total Booked + Current PnL: 20.60 L (16.4%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.33%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 86.21 L (59.18%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.83%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-6.16,76.0,X-LC,4.18,234920.0,24554.0,4229.0,1.28,11.67,1.80,13.68,37.0,5.81,1.63,30.12,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,111.31,65.0,M-SC,1.43,91071.0,-9706.0,9799.0,-2.93,-9.63,10.76,0.09,245.0,-0.99,0.63,19.65,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.0,-10.05,39.0,X-MC,2.76,88950.0,-1728.0,16634.0,-0.40,-1.91,18.70,16.44,101.0,-0.10,0.62,17.89,X40,ATH,PHARMA
42,ITC,452.0,-36.66,63.0,X-LC,1.42,204353.0,4215.0,16675.0,-0.00,2.11,8.16,10.44,4.0,0.25,1.42,7.79,X40,NTT,FMCG
85,VOLTAS,1530.0,1.02,52.0,X-MC,3.89,212055.0,20313.0,17452.0,-1.03,10.59,8.23,19.69,99.0,1.16,1.47,18.11,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,53.97,68.0,M-MC,10.14,242681.0,17719.0,149928.0,6.64,7.88,61.78,74.53,192.0,0.12,1.68,42.67,XY24,BTT,STEEL
82,VAIBHAVGBL,521.00,66.39,69.0,H-SC,3.80,144079.0,-38696.0,149770.0,6.48,-21.17,103.95,60.77,125.0,-0.26,1.00,29.05,XR,NTT,JEWELLERY
57,RECLTD,446.00,50.38,62.0,M-LC,5.46,210760.0,7590.0,34544.0,3.86,3.74,16.39,20.74,55.0,0.22,1.46,10.85,XY25,NTT,FINANCE
84,VBL,671.64,-17.33,56.0,X-LC,9.18,297882.0,-17960.0,130621.0,2.81,-5.69,43.85,35.67,5.0,-0.14,2.07,7.38,X40N,ATH,FMCG
32,HINDZINC,730.22,29.16,51.0,M-LC,11.37,210236.0,5160.0,107410.0,2.57,2.52,51.09,54.89,52.0,0.05,1.46,27.22,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,STARHEALTH,761.00,14.16,46.0,H-SC,12.48,254293.0,-26830.0,159696.0,-5.50,-9.54,62.80,47.26,144.0,-0.17,1.76,36.24,XY24,NTT,INSURANCE
17,CAMS,5211.76,-5.61,48.0,X-SC,0.41,107380.0,5376.0,38549.0,-3.77,5.27,35.90,43.06,122.0,0.14,0.74,23.29,X40N,ATH,MISC
79,TTKPRESTIG,770.00,111.31,65.0,M-SC,1.43,91071.0,-9706.0,9799.0,-2.93,-9.63,10.76,0.09,245.0,-0.99,0.63,19.65,OX40N,NTT,DURABLES
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88893.0,-39777.0,85071.0,-2.72,-30.91,95.70,35.20,219.0,-0.47,0.62,2.03,X40,NTT,FOOTWEAR
5,ANGELONE,3033.00,15.31,60.0,X-SC,5.45,203713.0,12707.0,45000.0,-2.11,6.65,22.09,30.21,157.0,0.28,1.41,28.92,X40N,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.0,-29.23,65.0,H-SC,16.15,201935.0,1981.0,95253.0,0.87,0.99,47.17,48.63,115.0,0.02,1.40,12.66,XR,NTT,MISC
87,WIPRO,420.0,-14.25,46.0,M-LC,5.72,150406.0,-539.0,109992.0,0.09,-0.36,73.13,72.51,53.0,-0.00,1.04,5.64,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,111.31,65.0,M-SC,1.43,91071.0,-9706.0,9799.0,-2.93,-9.63,10.76,0.09,245.0,-0.99,0.63,19.65,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.24,54.0,H-SC,2.73,226125.0,-43542.0,79867.0,0.70,-16.15,35.32,13.47,138.0,-0.55,1.57,14.98,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-22.20,54.0,H-MC,6.76,105174.0,-26661.0,68889.0,-0.17,-20.22,65.50,32.03,98.0,-0.39,0.73,17.80,OX40N,NTT,IT
18,CERA,9475.0,-24.82,38.0,H-SC,2.66,136976.0,-38927.0,80953.0,-0.19,-22.13,59.10,23.89,149.0,-0.48,0.95,18.30,OX40N,NTT,CERAMICS
68,SIS,528.0,2040.41,47.0,H-SC,3.15,86381.0,-24651.0,48261.0,-0.94,-22.20,55.87,21.26,156.0,-0.51,0.60,16.71,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.0,-15.53,75.0,H-MC,16.24,216689.0,36881.0,75538.0,-0.14,20.51,34.86,62.52,88.0,0.49,1.5,61.59,XR,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,53.97,68.0,M-MC,10.14,242681.0,17719.0,149928.0,6.64,7.88,61.78,74.53,192.0,0.12,1.68,42.67,XY24,BTT,STEEL
57,RECLTD,446.00,50.38,62.0,M-LC,5.46,210760.0,7590.0,34544.0,3.86,3.74,16.39,20.74,55.0,0.22,1.46,10.85,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.16,51.0,M-LC,11.37,210236.0,5160.0,107410.0,2.57,2.52,51.09,54.89,52.0,0.05,1.46,27.22,X5K,ATH,METALS
47,KPIGREEN,731.05,10.94,72.0,H-SC,5.37,132048.0,6751.0,52172.0,0.26,5.39,39.51,47.03,141.0,0.13,0.92,65.34,MH,ATH,POWER
80,UNIONBANK,163.00,-8.53,64.0,M-LC,9.41,166013.0,25173.0,19324.0,-0.72,17.87,11.64,31.59,66.0,1.30,1.15,49.10,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-15.53,75.0,H-MC,16.24,216689.0,36881.0,75538.0,-0.14,20.51,34.86,62.52,88.0,0.49,1.50,61.59,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-37.38,66.0,M-SC,3.39,129840.0,15042.0,78995.0,2.18,13.10,60.84,81.92,223.0,0.19,0.90,53.71,XR,NTT,DURABLES
47,KPIGREEN,731.05,10.94,72.0,H-SC,5.37,132048.0,6751.0,52172.0,0.26,5.39,39.51,47.03,141.0,0.13,0.92,65.34,MH,ATH,POWER
37,INDIAMART,4810.62,-53.33,56.0,H-SC,6.79,130518.0,7182.0,124449.0,-0.72,5.82,95.35,106.72,119.0,0.06,0.91,30.56,AR,ATH,MISC
32,HINDZINC,730.22,29.16,51.0,M-LC,11.37,210236.0,5160.0,107410.0,2.57,2.52,51.09,54.89,52.0,0.05,1.46,27.22,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,18.69,149374.0,-127149.0,237221.0,0.35,-45.98,158.81,39.81,54.0,-0.54,1.04,5.28,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88893.0,-39777.0,85071.0,-2.72,-30.91,95.70,35.20,219.0,-0.47,0.62,2.03,X40,NTT,FOOTWEAR
45,JSWINFRA,342.00,-20.28,38.0,X-MC,5.34,200171.0,-176.0,30686.0,0.54,-0.09,15.33,15.23,178.0,-0.01,1.39,32.13,X40N,NTT,REALTY
2,ABBOTINDIA,35195.00,-10.05,39.0,X-MC,2.76,88950.0,-1728.0,16634.0,-0.40,-1.91,18.70,16.44,101.0,-0.10,0.62,17.89,X40,ATH,PHARMA
24,DMART,5391.45,-16.66,39.0,X-LC,9.89,101964.0,-622.0,27428.0,0.65,-0.61,26.90,26.13,38.0,-0.02,0.71,24.81,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-2.18,50.0,H-LC,1.06,157340.0,-28755.0,76231.0,1.19,-15.45,48.45,25.51,15.0,-0.38,1.09,15.95,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,63.0,X-LC,1.42,204353.0,4215.0,16675.0,-0.00,2.11,8.16,10.44,4.0,0.25,1.42,7.79,X40,NTT,FMCG
53,PGHH,17907.65,-30.28,39.0,X-MC,2.57,201630.0,810.0,66981.0,0.33,0.40,33.22,33.76,80.0,0.01,1.40,5.62,X40,ATH,FMCG
29,HAVELLS,2069.16,-5.47,50.0,X-MC,3.03,315609.0,-12382.0,118921.0,1.51,-3.77,37.68,32.48,92.0,-0.10,2.19,7.22,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,1.02,52.0,X-MC,3.89,212055.0,20313.0,17452.0,-1.03,10.59,8.23,19.69,99.0,1.16,1.47,18.11,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88893.0,-39777.0,85071.0,-2.72,-30.91,95.70,35.20,219.0,-0.47,0.62,2.03,X40,NTT,FOOTWEAR
52,PAGEIND,51769.93,-27.18,41.0,X-MC,6.80,82210.0,-1300.0,21333.0,0.46,-1.56,25.95,23.98,82.0,-0.06,0.57,4.17,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-14.06,49.0,X-MC,5.92,180941.0,-19565.0,116725.0,1.21,-9.76,64.51,48.45,90.0,-0.17,1.25,4.34,X40N,ATH,FINANCE
21,COLPAL,3767.14,-3.45,49.0,X-MC,7.86,225530.0,-37835.0,151173.0,1.01,-14.37,67.03,43.04,84.0,-0.25,1.56,4.70,XY25,ATH,FMCG
3,ACC,3906.00,-37.20,56.0,X-MC,2.55,187160.0,-50791.0,203443.0,0.59,-21.35,108.70,64.15,174.0,-0.25,1.30,4.99,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-45.45,50.0,X-SC,38.22,54049.0,-10957.0,161882.0,0.22,-16.86,299.51,232.18,198.0,-0.07,0.37,5.43,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.18,41.0,X-MC,6.80,82210.0,-1300.0,21333.0,0.46,-1.56,25.95,23.98,82.0,-0.06,0.57,4.17,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.20,47.0,X-SC,4.81,82870.0,-62290.0,138219.0,1.46,-42.91,166.79,52.31,136.0,-0.45,0.57,10.93,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.05,39.0,X-MC,2.76,88950.0,-1728.0,16634.0,-0.40,-1.91,18.70,16.44,101.0,-0.10,0.62,17.89,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88893.0,-39777.0,85071.0,-2.72,-30.91,95.70,35.20,219.0,-0.47,0.62,2.03,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-27.72,56.0,X-LC,11.29,288317.0,-57639.0,124351.0,0.30,-16.66,43.13,19.28,1.0,-0.46,2.00,6.59,X40,ATH,IT
40,INFY,2275.00,-17.09,61.0,X-LC,6.13,322823.0,9935.0,161767.0,1.03,3.18,50.11,54.87,3.0,0.06,2.24,11.72,X40,BTT,IT
42,ITC,452.00,-36.66,63.0,X-LC,1.42,204353.0,4215.0,16675.0,-0.00,2.11,8.16,10.44,4.0,0.25,1.42,7.79,X40,NTT,FMCG
84,VBL,671.64,-17.33,56.0,X-LC,9.18,297882.0,-17960.0,130621.0,2.81,-5.69,43.85,35.67,5.0,-0.14,2.07,7.38,X40N,ATH,FMCG
1,ABB,7934.00,-39.28,57.0,H-LC,5.79,253728.0,-7891.0,127092.0,2.14,-3.02,50.09,45.57,7.0,-0.06,1.76,7.96,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7216.67,66.0,L-SC,7.65,80263.0,-13547.0,90167.0,1.19,-14.44,112.34,81.67,269.0,-0.15,0.56,55.43,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,27.92,81178.0,-32371.0,72419.0,-0.73,-28.51,89.21,35.27,268.0,-0.45,0.56,98.68,XR,NTT,HOTELS
50,MASFIN,398.61,-20.05,46.0,H-SC,9.15,90705.0,-7275.0,28880.0,0.22,-7.42,31.84,22.05,152.0,-0.25,0.63,31.85,XR,ATH,FINANCE
79,TTKPRESTIG,770.00,111.31,65.0,M-SC,1.43,91071.0,-9706.0,9799.0,-2.93,-9.63,10.76,0.09,245.0,-0.99,0.63,19.65,OX40N,NTT,DURABLES
86,WHIRLPOOL,2270.00,-37.38,66.0,M-SC,3.39,129840.0,15042.0,78995.0,2.18,13.10,60.84,81.92,223.0,0.19,0.90,53.71,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,-196.59,74.0,M-SC,29.66,166248.0,17028.0,127146.0,-1.36,11.41,76.48,96.62,208.0,0.13,1.15,79.92,XY25,NTT,FINANCE
69,SONACOMS,806.63,-31.93,72.0,M-SC,7.97,84700.0,-16459.0,56461.0,-0.07,-16.27,66.66,39.54,202.0,-0.29,0.59,19.44,AR,ATH,AUTO
12,BANKINDIA,190.00,-15.53,75.0,H-MC,16.24,216689.0,36881.0,75538.0,-0.14,20.51,34.86,62.52,88.0,0.49,1.50,61.59,XR,NTT,BANKS
47,KPIGREEN,731.05,10.94,72.0,H-SC,5.37,132048.0,6751.0,52172.0,0.26,5.39,39.51,47.03,141.0,0.13,0.92,65.34,MH,ATH,POWER
86,WHIRLPOOL,2270.00,-37.38,66.0,M-SC,3.39,129840.0,15042.0,78995.0,2.18,13.10,60.84,81.92,223.0,0.19,0.90,53.71,XR,NTT,DURABLES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.49
1,25,43.98
2,50,74.91


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.93
LC    32.37
MC    24.65
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.05
X40      16.26
XY25     12.09
XR       11.45
X40N     10.56
AR        7.83
OX40N     6.50
X200      1.76
X5K       1.46
SR        1.07
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.35
X-LC    21.63
X-MC    18.76
M-SC    12.65
M-LC     5.11
H-LC     4.61
X-SC     4.48
H-MC     3.89
M-MC     1.68
L-SC     1.45
L-LC     1.02
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.47,-3.34,37.06
IT,12.51,-16.44,78.90
FINANCE,12.08,-8.34,57.45
MISC,6.84,-16.39,80.89
ELECTRICAL,5.81,-7.85,46.09
BANKS,5.67,-4.50,66.25
PAINTS,5.50,-16.48,33.84
INSURANCE,3.73,-2.52,40.75
AUTO,2.77,-44.23,104.43


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3237626.0,22
XR,1295127.0,14
AR,1159594.0,9
X40,812489.0,12
X40N,635605.0,9
OX40N,546094.0,9
XY25,438410.0,8
SR,266444.0,2
X5K,107410.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3317498.0,24
M-SC,1218424.0,15
X-MC,1171993.0,14
X-LC,1020708.0,13
X-SC,532630.0,6
H-MC,319201.0,3
H-LC,273842.0,3
M-LC,271270.0,4
L-SC,251809.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1158079.0      6
           AR         832818.0      5
           XR         763950.0      7
M-SC       XY24       697660.0      6
X-MC       XY24       571982.0      4
X-LC       X40        443443.0      5
           XY24       303997.0      2
X-SC       X40N       285677.0      4
X-MC       X40        283975.0      6
H-SC       SR         266444.0      2
           OX40N      244035.0      3
H-LC       AR         203323.0      2
X-LC       X40N       202517.0      3
H-MC       XY24       174774.0      1
X-MC       XY25       168625.0      2
L-SC       XR         162586.0      2
X-SC       XY24       161882.0      1
M-MC       XY24       149928.0      1
X-MC       X40N       147411.0      2
M-SC       OX40N      143947.0      4
           XY25       127146.0      1
           XR         126218.0      2
           AR         123453.0      2
M-LC       XR         109992.0      1
           X5K        107410.0      1
L-SC       OX40N       89223.0      1
X-SC       X40         85071.0      1
H-MC       XR          75538.0      1
X-LC       XY25        70751.0      3
H-LC       X200        70519.0      1
H-MC       OX40N       68889.0      1
L-MC       XR          56843.0      1
H-SC       MH          52172.0      1
L-LC       XY25        37344.0      1
M-LC       XY25        34544.0      1
           XY24        19324.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 50.0 seconds
